In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import pandas as pd
import time
import re

pd.options.mode.chained_assignment = None

In [ ]:
df_all = pd.read_csv('./data/제주특별자치도음식점목록(통합).csv')
df_all = df_all[df_all.영업상태명 != '폐업']
df_all.dropna(subset=['소재지전체주소'], inplace=True)
regex = re.compile(r'\s\w+\s')
df_all['시군'] = df_all['소재지전체주소'].apply(lambda x: regex.search(x).group().strip())
df_all = df_all.loc[df_all['업종구분소분류']=='한식',['시군','업종구분대분류','업종구분소분류','사업장명']]
df_all.reset_index(drop=True,inplace=True)
df_all['시도']='제주도'
print(df_all['업종구분대분류'].value_counts())
df_all['별점'] = np.NaN
df_all['방문자 리뷰수'] = np.NaN
df_all['블로그 리뷰수'] = np.NaN
df_all.rename(columns={'업종구분소분류': '업종','사업장명': '업소명'}, inplace=True)
df_all = df_all[['시도','시군','업종','업소명','별점','방문자 리뷰수','블로그 리뷰수']]
df_all

일반음식점    6792
Name: 업종구분대분류, dtype: int64


,시도,시군,업종,업소명,별점,방문자 리뷰수,블로그 리뷰수
0,제주도,제주시,한식,일품순두부한림점,NaN,NaN,NaN
1,제주도,제주시,한식,연두랑고갈비,NaN,NaN,NaN
2,제주도,제주시,한식,길섶나그네,NaN,NaN,NaN
3,제주도,제주시,한식,대붕정,NaN,NaN,NaN
4,제주도,제주시,한식,쉬멍해장국,NaN,NaN,NaN
...,...,...,...,...,...,...,...
6787,제주도,서귀포시,한식,빨강아구찜,NaN,NaN,NaN
6788,제주도,서귀포시,한식,벙커포차,NaN,NaN,NaN
6789,제주도,서귀포시,한식,온달수제돈가스,NaN,NaN,NaN
6790,제주도,서귀포시,한식,일품양평해장국,NaN,NaN,NaN


In [ ]:
driver = webdriver.Chrome('./chromedriver.exe')

C:\Users\ASUS\AppData\Local\Temp\ipykernel_34812\913636808.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')


In [ ]:
#평점, 방문자리뷰수, 블로그리뷰수 크롤링
def rank(df, i):
    data = df['시군'][i] + ' ' + df['업소명'][i]    #검색할 가게 이름
    driver.get("https://map.naver.com/v5/search/"+data)    #검색창에 가게이름 입력
    time.sleep(2)
    driver.implicitly_wait(2)
    driver.switch_to.frame('searchIframe')    #가게 정보에 접근하기 위해 iframe으로 스위치
    driver.implicitly_wait(2)
    try:    #가게 정보가 없을 경우 except 예외처리
        temp = driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul')
        driver.implicitly_wait(2)
        button = temp.find_elements(By.TAG_NAME,'a')
        driver.implicitly_wait(2)
        if '이미지수' in button[0].text or button[0].text == '':    #검색 후 해당 가게 정보창이 바로 안뜨고 가게들 리스트가 뜰 경우 맨 위 가게 정보로 이동
            button[1].send_keys(Keys.ENTER)
        else:
            button[0].send_keys(Keys.ENTER)
        driver.implicitly_wait(2)
        time.sleep(2)
        driver.switch_to.default_content()
        try:
            driver.switch_to.frame('entryIframe')    #제주시 복집에서 에러 발생하여 코드 추가
        except:
            pass
        driver.implicitly_wait(2)
        try:    #별점, 방문자, 블로그 리뷰 있을때
            rank = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/em')
            visit = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a/em')
            blog = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[3]/a/em')
            print(f'{i}. 음식점 이름:{data}, 별점:{rank.text}, 방문자 리뷰수:{visit.text}, 블로그 리뷰수:{blog.text}')
            df['별점'][i] = rank.text
            df['방문자 리뷰수'][i] = visit.text
            df['블로그 리뷰수'][i] = blog.text
            review(df,i)
        except NoSuchElementException:
            try:    #별점, 방문자 리뷰만 있을때
                rank = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/em')
                visit = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a/em')
                print(f'{i}. 음식점 이름:{data}, 별점:{rank.text}, 방문자 리뷰수:{visit.text}, 블로그 리뷰수:0')
                df['별점'][i] = rank.text
                df['방문자 리뷰수'][i] = visit.text
                review(df,i)
            except NoSuchElementException:
                try:    #방문자, 블로그 리뷰만 있을때
                    visit = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/a/em')
                    blog = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a/em')
                    print(f'{i}. 음식점 이름:{data}, 별점:N/A, 방문자 리뷰수:{visit.text}, 블로그 리뷰수:{blog.text}')
                    df['방문자 리뷰수'][i] = visit.text
                    df['블로그 리뷰수'][i] = blog.text
                    review(df,i)
                except NoSuchElementException:
                    try:    #방문자 리뷰만 있을때
                        visit = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span/a')
                        print(f'{i}. 음식점 이름:{data}, 별점:N/A, 방문자 리뷰수:{visit.text}, 블로그 리뷰수:0')
                    except NoSuchElementException:
                        print(f'{i}. 음식점 이름:{data} 패스')
    except NoSuchElementException:
        print(f'{i}. 음식점 이름:{data} 패스')


def review(df,idx):    #키워드리뷰 크롤링
    for i in range(1,10):    #리뷰 메뉴바 탐색
        nav_review = driver.find_element(By.XPATH,f'//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[{i}]/span')
        if nav_review.text == "리뷰" :
            nav_review.click()
            break
    while True:    #키워드리뷰 접힌 부분 다 보일때까지 확장
        try:
            driver.find_element(By.XPATH,'(//*[local-name()="svg" and @xmlns="http://www.w3.org/2000/svg" and @class="q1Lf9"])[2]').click()
        except NoSuchElementException:
            break
    for i in range(1,25):    #키워드리뷰와 개수 크롤링
        try:
            review = driver.find_element(By.XPATH,f'//*[@id="app-root"]/div/div/div/div[7]/div[3]/div[1]/div/div/div[2]/ul/li[{i}]/div[2]/span[1]')
            review_cnt = driver.find_element(By.XPATH,f'//*[@id="app-root"]/div/div/div/div[7]/div[3]/div[1]/div/div/div[2]/ul/li[{i}]/div[2]/span[2]')
            review_cnt = re.sub(r'^.*\n', '', review_cnt.text)
            if i == 1:    #첫번째 키워드리뷰 확인
                print(review.text,review_cnt)
            if review.text in df.columns:    #df에 해당 키워드 컬럼이 있으면 값 추가, 없으면 열 생성후 값 추가
                df[review.text][idx] = review_cnt
            else:
                df[review.text] = 0
                df[review.text][idx] = review_cnt
        except NoSuchElementException:    #키워드리뷰 XPath값 다를시 예외처리
            try:
                review = driver.find_element(By.XPATH,f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[1]/div/div/div[2]/ul/li[{i}]/div[2]/span[1]')
                review_cnt = driver.find_element(By.XPATH,f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[1]/div/div/div[2]/ul/li[{i}]/div[2]/span[2]')
                review_cnt = re.sub(r'^.*\n', '', review_cnt.text)
                if i == 1:
                    print(review.text,review_cnt)
                if review.text in df.columns:
                    df[review.text][idx] = review_cnt
                else:
                    df[review.text] = 0 #열 생성후 0으로 초기화
                    df[review.text][idx] = review_cnt
            except NoSuchElementException:
                break

In [ ]:
for i in range(len(df_all)):
    rank(df_all, i)
    if i % 5 == 0:    #5개 검색시마다 csv파일로 저장
        df_all.to_csv('./data/rank_review_test.csv', encoding="utf-8-sig")

df_all.to_csv('./data/rank_review_test.csv', encoding="utf-8-sig")

0. 음식점 이름:제주시 일품순두부한림점, 별점:4.37, 방문자 리뷰수:378, 블로그 리뷰수:33
"음식이 맛있어요" 93
1. 음식점 이름:제주시 연두랑고갈비, 별점:N/A, 방문자 리뷰수:28, 블로그 리뷰수:2
"음식이 맛있어요" 25
2. 음식점 이름:제주시 길섶나그네, 별점:5, 방문자 리뷰수:22, 블로그 리뷰수:51
"음식이 맛있어요" 13
3. 음식점 이름:제주시 대붕정 패스
4. 음식점 이름:제주시 쉬멍해장국, 별점:4.1, 방문자 리뷰수:147, 블로그 리뷰수:169
"음식이 맛있어요" 39


KeyboardInterrupt: 